### Initialization

### This notebook demostrates Lake Formation Governed Tables ACID Transactions behavior in below scenarios:
  Scenario A: User A makes changes using T1 and commit transaction. User B queries table using T2 and see changes.<br>
  Scenario B: User A makes changes using T3 but does not commit transaction. User B queries using T4 and cannot see the changes.<br>
  Scenario C: User A commits transaction T3, User B again queries using T4 but still cannot see the changes since T4 started before T3 is committed.<br>

In [1]:
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from pyspark.sql import DataFrame, Row
from awsglue.dynamicframe import DynamicFrame
from awsglue import DynamicFrame


glueContext = GlueContext(SparkContext.getOrCreate())
spark = glueContext.spark_session
job = Job(glueContext)

# Please replace below with your parameters
bucket_name = "lf-workshop-162611428811"
database_name = "governed_demo"
table_name = "employee_details"

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
13,application_1631086109765_0014,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Begin Transaction (tcampId1)

In [2]:
tcampId1 = glueContext.begin_transaction(read_only=False)
print(tcampId1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

7bcf6c915e94438ea608b4d209bf2651

### Check initial rows for employees from Denver

In [3]:
dyf1 = glueContext.create_dynamic_frame.from_catalog(
    database = database_name, 
    table_name = table_name, 
    transformation_ctx = "dyf1", 
    additional_options = {
        "transactionId": tcampId1
    }
)
df1 = dyf1.toDF()
df11 = df1.select('employee_id','first_name','last_name','age','city','salary')
df11.where("city == 'Denver'").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+----------+---------+---+------+------+
|employee_id|first_name|last_name|age|  city|salary|
+-----------+----------+---------+---+------+------+
|       1013|   William|    Moore| 37|Denver|  3998|
|       1018| Frederick|   Wilson| 38|Denver|  4500|
+-----------+----------+---------+---+------+------+

### Commit Transaction (tcampId1)

In [4]:
glueContext.commit_transaction(tcampId1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

JavaObject id=o107

### Data Analyst begin transaction (tcampId2)

In [5]:
tcampId2 = glueContext.begin_transaction(read_only=False)
print(tcampId2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

f795b58b491743e19638c38f5f97917c

### Find  employees from Denver after DB Admin commits Transaction (txId1)

In [6]:
dyf2 = glueContext.create_dynamic_frame.from_catalog(
    database = database_name, 
    table_name = table_name, 
    transformation_ctx = "dyf2", 
    additional_options = {
        "transactionId": tcampId2
    }
)
df2 = dyf2.toDF()
df3 = df2.select('employee_id','first_name','last_name','age','city','salary')
df3.where("city == 'Denver'").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+----------+---------+---+------+------+
|employee_id|first_name|last_name|age|  city|salary|
+-----------+----------+---------+---+------+------+
|       1013|   William|    Moore| 37|Denver|  3998|
|       1018| Frederick|   Wilson| 38|Denver|  4500|
|       1022|     Ellis|      Dow| 40|Denver|  4250|
+-----------+----------+---------+---+------+------+

### Data Analyst commit transaction (tcampId2)

In [7]:
glueContext.commit_transaction(tcampId2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

JavaObject id=o124

### Data Analyst begin transaction (tcampId3)

In [8]:
tcampId3 = glueContext.begin_transaction(read_only=False)
print(tcampId3)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

22108a3b3aed4372ae50cde28864a1b7

### Find employees from Denver after DB Admin adds rows for new employees but not commit its transaction (txId2)

In [9]:
dyf3 = glueContext.create_dynamic_frame.from_catalog(
    database = database_name, 
    table_name = table_name, 
    transformation_ctx = "dyf3", 
    additional_options = {
        "transactionId": tcampId3
    }
)
df4 = dyf3.toDF()
df5 = df4.select('employee_id','first_name','last_name','age','city','salary')
df5.where("city == 'Denver'").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+----------+---------+---+------+------+
|employee_id|first_name|last_name|age|  city|salary|
+-----------+----------+---------+---+------+------+
|       1013|   William|    Moore| 37|Denver|  3998|
|       1018| Frederick|   Wilson| 38|Denver|  4500|
|       1022|     Ellis|      Dow| 40|Denver|  4250|
+-----------+----------+---------+---+------+------+

### Find employees from Denver after DB Admin adds rows for new employees and committed its transaction (txId2). But Data Analyst started transaction tcampId2 before txId2 was committed.

In [10]:
dyf4 = glueContext.create_dynamic_frame.from_catalog(
    database = database_name, 
    table_name = table_name, 
    transformation_ctx = "dyf4", 
    additional_options = {
        "transactionId": tcampId3
    }
)
df6 = dyf4.toDF()
df7 = df6.select('employee_id','first_name','last_name','age','city','salary')
df7.where("city == 'Denver'").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+----------+---------+---+------+------+
|employee_id|first_name|last_name|age|  city|salary|
+-----------+----------+---------+---+------+------+
|       1013|   William|    Moore| 37|Denver|  3998|
|       1018| Frederick|   Wilson| 38|Denver|  4500|
|       1022|     Ellis|      Dow| 40|Denver|  4250|
+-----------+----------+---------+---+------+------+

### Data Analyst commit transaction (tcampId3)

In [11]:
glueContext.commit_transaction(tcampId3)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

JavaObject id=o157

### Data Analyst begin transaction (tcampId4)

In [12]:
tcampId4 = glueContext.begin_transaction(read_only=False)
print(tcampId4)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

cd90166c6b7842dfbf08639437c07f0e

### Find employees from Denver after DB Admin commits Transaction (txId2)

In [13]:
dyf5 = glueContext.create_dynamic_frame.from_catalog(
    database = database_name, 
    table_name = table_name, 
    transformation_ctx = "dyf5", 
    additional_options = {
        "transactionId": tcampId4
    }
)
df8 = dyf5.toDF()
df9 = df8.select('employee_id','first_name','last_name','age','city','salary')
df9.where("city == 'Denver'").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+----------+---------+---+------+------+
|employee_id|first_name|last_name|age|  city|salary|
+-----------+----------+---------+---+------+------+
|       1013|   William|    Moore| 37|Denver|  3998|
|       1018| Frederick|   Wilson| 38|Denver|  4500|
|       1022|     Ellis|      Dow| 40|Denver|  4250|
|       1024|     Megan|    Sisco| 39|Denver|  4320|
+-----------+----------+---------+---+------+------+

### Data Analyst commit transaction (tcampId3)

In [14]:
glueContext.commit_transaction(tcampId4)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

JavaObject id=o174

### Time Travel query example

In [15]:
dyf6 = glueContext.create_dynamic_frame.from_catalog(
    database = database_name, 
    table_name = table_name, 
    transformation_ctx = "dyf6", 
    additional_options = {
        "asOfTime": "2021-09-15 05:59:38"
    }
)
df10 = dyf6.toDF()
df11 = df10.select('employee_id','first_name','last_name','age','city','salary')
df11.where("city == 'Denver'").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+----------+---------+---+------+------+
|employee_id|first_name|last_name|age|  city|salary|
+-----------+----------+---------+---+------+------+
|       1013|   William|    Moore| 37|Denver|  3998|
|       1018| Frederick|   Wilson| 38|Denver|  4500|
|       1022|     Ellis|      Dow| 40|Denver|  4250|
+-----------+----------+---------+---+------+------+